In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

## client profile data

In [ ]:
df_client = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_demo.txt')
df_client.head(2)
df_client.columns

In [ ]:
# clean column names

In [ ]:
# check column : bal , balance? 

In [ ]:
df_client.shape

In [ ]:
df_client.dtypes

In [ ]:
df_client.duplicated().sum()

In [ ]:
df_client['client_id'].duplicated().sum()

In [ ]:
# handle null values
df_client.isna().sum()

In [ ]:
df_client.nunique()

In [ ]:
df_client['clnt_tenure_mnth'].value_counts()

In [ ]:
df_client['clnt_tenure_yr'].value_counts()

In [ ]:
# check values : U, X
df_client['gendr'].value_counts()

In [ ]:
df_client['num_accts'].value_counts()

In [ ]:
df_client['calls_6_mnth'].value_counts()



In [ ]:
df_client['logons_6_mnth'].value_counts()


In [ ]:
df_client.describe()

## Digital Footprints data

In [ ]:
df_footprints_1 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_1.txt')
df_footprints_2 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_2.txt')

df_footprints_1.columns, df_footprints_2.columns

In [ ]:
df_footprints_1.dtypes, df_footprints_2.dtypes

In [ ]:
# date_time column, object dtype ---> datetime dtype(each value is datetime object)
df_footprints_1['date_time'] = pd.to_datetime(df_footprints_1['date_time'],errors = 'coerce')
df_footprints_1['date_time'].isna().sum()
df_footprints_2['date_time'] = pd.to_datetime(df_footprints_2['date_time'],errors = 'coerce')
df_footprints_2['date_time'].isna().sum()

In [ ]:
df_footprints_1.shape, df_footprints_2.shape

In [ ]:
# handle duplicate rows 
df_footprints_1.duplicated().sum(), df_footprints_2.duplicated().sum()

In [ ]:
df_footprints_1.isna().sum(), df_footprints_2.isna().sum()

In [ ]:
df_footprints_1.nunique(), df_footprints_2.nunique()

In [ ]:
df_footprints_1['process_step'].unique(), df_footprints_2['process_step'].unique()

## Experiment data

In [ ]:

df_experiments = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_experiment_clients.txt')

df_experiments.columns, df_experiments.shape,df_experiments.dtypes


In [ ]:
df_experiments.head(2)

In [ ]:
df_experiments.duplicated().sum()

In [ ]:
df_experiments['client_id'].duplicated().sum()

In [ ]:
# handle null values in column 'Variation'
df_experiments.isna().sum()

In [ ]:
df_experiments['Variation'].value_counts(normalize=True)